In [ ]:
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
from chronos import ChronosPipeline
import torch


In [ ]:
START = "2020-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace=True)
    return data

data = load_data('BTC-USD')
df = data.drop(['Date', 'Adj Close'], axis=1)

train = df[0:int(len(df)*0.70)]
test = df[int(len(df)*0.70):]

scaler = MinMaxScaler(feature_range=(0, 1))
train_close = scaler.fit_transform(train.iloc[:, 4:5].values)
test_close = scaler.transform(test.iloc[:, 4:5].values)


Train LSTM model

In [ ]:
x_train = []
y_train = []

for i in range(100, len(train_close)):
    x_train.append(train_close[i-100:i, 0])
    y_train.append(train_close[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation="linear"))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train, batch_size=32, epochs=50)

model.save('model.h5')


use Chronos

In [ ]:
saved_model = tf.keras.models.load_model('model.h5')

# Wrap the model with ChronosPipeline
pipeline = ChronosPipeline(saved_model)

# Prepare input data for forecasting
past_100_days = pd.DataFrame(train_close[-100:])
test_df = pd.DataFrame(test_close)
final_df = pd.concat([past_100_days, test_df], ignore_index=True)

input_data = scaler.fit_transform(final_df)

x_test = []
y_test = []
for i in range(100, input_data.shape[0]):
    x_test.append(input_data[i-100:i])
    y_test.append(input_data[i, 0])

x_test, y_test = np.array(x_test), np.array(y_test)

# Perform forecasting with Chronos
forecast_horizon = 10  # Adjust as needed
num_samples = 100  # Adjust as needed

forecast = pipeline.predict(
    context=torch.tensor(input_data[-100:]),  # Provide recent context for forecasting
    prediction_length=forecast_horizon,
    num_samples=num_samples,
)

# Optionally, print or save the forecast results
print(forecast.shape)  # Example of printing forecast shape

# Plotting the results
scaling_factor = scaler.scale_[0]
scale_factor = 1 / scaling_factor

y_pred = forecast * scale_factor
y_test = y_test[-forecast_horizon:] * scale_factor

plt.figure(figsize=(12, 6))
plt.plot(y_test, 'b', label="Original Price")
plt.plot(y_pred.mean(axis=1), 'r', label="Predicted Price")
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()
